In [1]:
import pandas as pd
import json

data = pd.read_csv('text2cypher_questions.csv')

/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data.head()

,question,type,database
0,What are the top 5 movies with a runtime great...,Simple Retrieval Queries,recommendations
1,List the first 3 directors born before 1950.,Simple Retrieval Queries,recommendations
2,Which 5 users have rated more than 20 movies?,Simple Retrieval Queries,recommendations
3,Identify the top 5 actors who have acted in mo...,Simple Retrieval Queries,recommendations
4,What are the top 3 genres associated with movi...,Simple Retrieval Queries,recommendations


In [3]:
data = data[data['database'] != 'slack']

In [4]:
schemas_df = pd.read_csv('text2cypher_schemas.csv')
schemas_df.head()
schemas = {}
for i, row in schemas_df.iterrows():
    schemas[row['database']] = row['schema']

In [5]:
# Create OpenAI Batch processing JSONL file

system = """Given an input question, convert it to a Cypher query.
To translate a question into a Cypher query, please follow these steps:

1. Carefully analyze the provided graph schema to understand what nodes, relationships, and properties are available. Pay attention to the node labels, relationship types, and property keys.

2. Identify the key entities and relationships mentioned in the natural language question. Map these to the corresponding node labels, relationship types, and properties in the graph schema.

3. Think through how to construct a Cypher query to retrieve the requested information step-by-step. Focus on:
   - Identifying the starting node(s) 
   - Traversing the necessary relationships
   - Filtering based on property values
   - Returning the requested information
Feel free to use multiple MATCH, WHERE, and RETURN clauses as needed.

4. Once you have finished constructing the Cypher query, provide the final query inside triple backticks ```cypher```.

5. Explain how your Cypher query will retrieve the necessary information from the graph to answer the original question. Provide this explanation inside <explanation> tags.

Remember, the goal is to construct a Cypher query that will retrieve the relevant information to answer the question based on the given graph schema. Carefully map the entities and relationships in the question to the nodes, relationships, and properties in the schema.

If the question cannot be answered by the information in the graph schema, say "Based on the given graph schema, there is not enough information to answer this question." inside ```cypher``` tags. Do not attempt to construct a query if the schema does not support it. """


def create_line(question, database, id):
    schema = schemas[database]
    user_message = f"""Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question:
    {schema}
    
    Question: {question}
    Cypher query:"""

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user_message},
    ]
    line = {
        "custom_id": "request-" + str(id),
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o",
            "messages": messages,
            "max_tokens": 1000,
            "temperature": 0,
        },
    }
    return line


with open("text2cypher_batch_input.jsonl", "w") as outfile:
    for i, q in data.iterrows():
        line = create_line(q["question"], q["database"], i)
        json.dump(line, outfile)
        outfile.write("\n")